# 1. Setup

## 1.1. Installing and importing libraries to the collab new session (therefore new environment)

In [2]:
!python -m nltk.downloader 'punkt'
#restart the runtime once installed

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
!pip install sumy

     |████████████████████████████████| 83 kB 1.5 MB/s 
     |████████████████████████████████| 10.1 MB 8.3 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=cda1de4cda7a78ac4e2b731df26160a3f220cb046189a3e131397ff63014f904
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=e9200c1f34a888a7d5c26a6ec9b0842e2da081fd7d742c214acdac514497b6bd
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry


In [3]:
!pip install datasets

     |████████████████████████████████| 542 kB 5.2 MB/s 
     |████████████████████████████████| 243 kB 10.4 MB/s 
     |████████████████████████████████| 118 kB 9.7 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 76 kB 3.6 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [4]:
!pip install rouge_score

In [5]:
# Importing necessary librairies
import datasets
from datasets import load_metric
from google.colab import drive
import nltk
import numpy as np
import pandas as pd
import pickle 
from sklearn.model_selection import train_test_split
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer


## 1.2. Importing personal drive to file in order to load the data


In [6]:
# Mount Google Drive to this Notebook instance.
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Loading the testing dataset

In [7]:
test_dataset = pd.read_csv('/content/drive/MyDrive/Data/test_dataset.csv') #this will be used to generate predictions 
df = pd.read_pickle('/content/drive/MyDrive/Data/SummaDevDocs_preprocessed.pickle') #this will be used to compute the ROUGE scores of the generated summaries 

In [8]:
test_dataset = test_dataset.groupby('document_label').agg(list)
test_dataset = test_dataset.reset_index()

In [9]:
#creating an empty list 
documents = []

#looping though the document labels in the df_summaries 
for label in test_dataset['document_label']:
  #looping through both the indexes and the text of the original summaries in the initial dataframe
  for index, docs in zip(df.index, df['document_text']):
        #setting the condition that if the index of the actual summary in the original document equals 
        #to the label of the document in the results (i.e. summaries) dataframe 
        #append the summary to the empty list
        if label == index:
            documents.append(docs)

test_dataset['document_text'] = documents

#creataing an empty list 
reference_summaries = []

#looping though the document labels in the df_summaries 
for label in test_dataset['document_label']:
  #looping through both the indexes and the text of the original summaries in the initial dataframe
  for index, summ in zip(df.index, df['summary_text']):
        #setting the condition that if the index of the actual summary in the original document equals 
        #to the label of the document in the results (i.e. summaries) dataframe 
        #append the summary to the empty list
        if label == index:
            reference_summaries.append(summ)

test_dataset['reference_summary'] = reference_summaries

test_dataset

,document_label,sentence,label,document_text,reference_summary
0,17,[The Africa Clean Energy (ACE) Technical Assis...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",The Africa Clean Energy (ACE) Technical Assist...,The Africa Clean Energy (ACE) Technical Assist...
1,18,[DFID’s Africa Regional Department (ARD) is th...,"[0, 0, 0, 0, 0, 0, 1, 0, 0]",DFID’s Africa Regional Department (ARD) is the...,DFID has engaged Coffey to provide third party...
2,19,[The project is working with the Federal Gover...,"[0, 1, 0, 0, 0, 0, 0, 0]",The project is working with the Federal Govern...,The project is working with the Federal Govern...
3,20,[This five-year project (September 2016 – Augu...,"[0, 0, 0, 1, 0, 0, 0, 0]",This five-year project (September 2016 – Augus...,Four major inter-related causes of conflict an...
4,36,"[PROJECT OBJECTIVES:, nnThe objective of the p...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",PROJECT OBJECTIVES:\n\nThe objective of the pr...,The objective of the project is to increase ba...
...,...,...,...,...,...
569,2967,[Money Trail is a joint project initiated by F...,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",Money Trail is a joint project initiated by Fr...,Money Trail is a joint project initiated by Fr...
570,2973,"[Gisha, Legal Center for Freedom of Movement i...","[0, 0, 0, 1, 0, 0, 0, 0]","Gisha\, Legal Center for Freedom of Movement i...",Gisha#s direct intervention with Palestinians ...
571,2976,[Legal Aid Service Providers Network Uganda (L...,"[0, 1, 1, 0, 0, 0, 0]",Legal Aid Service Providers Network Uganda (LA...,"this Voice linking and learning project, LASP..."
572,2977,[The latest findings from a countrywide season...,"[0, 0, 0, 0, 1, 0, 0, 0]",The latest findings from a countrywide seasona...,nOxfam provides integrated EFSVL (Emergency Fo...


# 3. Loading the summariser 

In [10]:
# Defining a function to parse, tokenize and summarise the text 
def summariser(doc):
  parser = PlaintextParser.from_string(doc, Tokenizer('english'))
  lexrank = LexRankSummarizer()
  summary = lexrank(parser.document, 2)
  return summary

# Applying the function to the dataset
#creating an empty list
lex_rank_summaries = []

#looping in the texts of teh datasets
for doc in test_dataset['document_text']:
  #applying the function to each text 
  #appending the result to the list 
  lex_rank_summaries.append(summariser(doc))

# Creating a column filled with the generated summaries 
test_dataset['predicted_summaries'] = lex_rank_summaries
test_dataset = test_dataset.reset_index()
test_dataset

,index,document_label,sentence,label,document_text,reference_summary,predicted_summaries
0,0,17,[The Africa Clean Energy (ACE) Technical Assis...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",The Africa Clean Energy (ACE) Technical Assist...,The Africa Clean Energy (ACE) Technical Assist...,(The ACE TAF is part of the broader ACE Busine...
1,1,18,[DFID’s Africa Regional Department (ARD) is th...,"[0, 0, 0, 0, 0, 0, 1, 0, 0]",DFID’s Africa Regional Department (ARD) is the...,DFID has engaged Coffey to provide third party...,(DFID’s Africa Regional Department (ARD) is th...
2,2,19,[The project is working with the Federal Gover...,"[0, 1, 0, 0, 0, 0, 0, 0]",The project is working with the Federal Govern...,The project is working with the Federal Govern...,(Focusing on strengthening customs operations ...
3,3,20,[This five-year project (September 2016 – Augu...,"[0, 0, 0, 1, 0, 0, 0, 0]",This five-year project (September 2016 – Augus...,Four major inter-related causes of conflict an...,(The project has a budget allocation of 8.2 mi...
4,4,36,"[PROJECT OBJECTIVES:, nnThe objective of the p...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",PROJECT OBJECTIVES:\n\nThe objective of the pr...,The objective of the project is to increase ba...,(To this aim the project proposes to modernize...
...,...,...,...,...,...,...,...
569,569,2967,[Money Trail is a joint project initiated by F...,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",Money Trail is a joint project initiated by Fr...,Money Trail is a joint project initiated by Fr...,(Finance Uncovered provides training in financ...
570,570,2973,"[Gisha, Legal Center for Freedom of Movement i...","[0, 0, 0, 1, 0, 0, 0, 0]","Gisha\, Legal Center for Freedom of Movement i...",Gisha#s direct intervention with Palestinians ...,(It has over a decade of experience in legal a...
571,571,2976,[Legal Aid Service Providers Network Uganda (L...,"[0, 1, 1, 0, 0, 0, 0]",Legal Aid Service Providers Network Uganda (LA...,"this Voice linking and learning project, LASP...","(On this Voice linking and learning project\, ..."
572,572,2977,[The latest findings from a countrywide season...,"[0, 0, 0, 0, 1, 0, 0, 0]",The latest findings from a countrywide seasona...,nOxfam provides integrated EFSVL (Emergency Fo...,(The latest findings from a countrywide season...


# 4. Evaluation

## 4.1. Data Preparation

In [11]:
# Creating lists of the reference and the predicted summaries
reference_summaries = test_dataset['reference_summary']
predicted_summaries = test_dataset['predicted_summaries']

# Converting the predicted summaries to strings 
predicted_summaries = [str(sent) for sent in predicted_summaries]

# Defining a function to clean the predicted sentences from the added format generated by text_rank
def cleaner(sent):
  sent = sent.replace("(<Sentence: ", "")
  sent = sent.replace(" <Sentence:", "")
  sent = sent.replace ('>)', "")
  sent = sent.replace('>,', "")
  return sent 

# Applying the function to both the predicted and the reference summaries for consistency
predicted_summaries = [cleaner(sent) for sent in predicted_summaries]
reference_summaries = [cleaner(sent) for sent in reference_summaries]

## 4.2. Computing the ROUGE scores

In [12]:
# Loading the ROUGE metric from Hugginface 
metric = load_metric('rouge')
rouge_scores = metric.compute(predictions=predicted_summaries, references=reference_summaries)
rouge_scores

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

{'rouge1': AggregateScore(low=Score(precision=0.35433618739257167, recall=0.7355361257946275, fmeasure=0.4518733694104298), mid=Score(precision=0.3720729564644151, recall=0.7590404849731309, fmeasure=0.47163565284748504), high=Score(precision=0.39096481779605646, recall=0.7804207168099796, fmeasure=0.4913218099274715)),
 'rouge2': AggregateScore(low=Score(precision=0.27887071048442835, recall=0.5818314866066446, fmeasure=0.35727118275424585), mid=Score(precision=0.30082112170144537, recall=0.6171793353997307, fmeasure=0.3826432154556109), high=Score(precision=0.32367633834644643, recall=0.6508750977941132, fmeasure=0.4079836867174939)),
 'rougeL': AggregateScore(low=Score(precision=0.32183280603643527, recall=0.6666210605271747, fmeasure=0.410559678855516), mid=Score(precision=0.34213469138600716, recall=0.6975681037111647, fmeasure=0.43424640166364226), high=Score(precision=0.3647639112884269, recall=0.7263917967016085, fmeasure=0.45822981670519664)),
 'rougeLsum': AggregateScore(low=

## 4.3. View a few examples

In [ ]:
print('This is the reference summary: ')
print(reference_summaries[16])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[16])

This is the reference summary: 
This project contributes to the financing of the expansion of the Koudougou university campus consisting of the construction and the equipment of new facilities
   
This is the predicted summary: 
This contributes to recurrent classroom management issues within the university as students have to share the available free classrooms with other faculties. Hence, in April 2017, the government requested OFID to contribute to the financing of the expansion of the university campus consisting of the construction and the equipment of new facilities including: a) Dormitories, b) Science and Technology Training and Academic Research Unit.


In [ ]:
print('This is the reference summary: ')
print(reference_summaries[69])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[69])

This is the reference summary: 
The development objective of the Remote Rural Communities Development Project for Bhutan is to increase agriculture productivity and access to community assets in remote rural areas by improving access to markets, irrigation, agricultural technologies, and community infrastructure.
   
This is the predicted summary: 
The development objective of the Remote Rural Communities Development Project for Bhutan is to increase agriculture productivity and access to community assets in remote rural areas by improving access to markets, irrigation, agricultural technologies, and community infrastructure. This component supports project management, including the monitoring and evaluation functions of the project.


In [ ]:
print('This is the reference summary: ')
print(reference_summaries[103])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[103])

This is the reference summary: 
The development objective Casablanca Municipal Support Program-for-Results Project for Morocco is to increase the investment capacity of the Municipality of Casablanca, improve the business environment in the Municipality of Casablanca, and enhance access to basic services in the Program Area. 
   
This is the predicted summary: 
The development objective Casablanca Municipal Support Program-for-Results Project for Morocco is to increase the investment capacity of the Municipality of Casablanca, improve the business environment in the Municipality of Casablanca, and enhance access to basic services in the Program Area. The Program activities are grouped under three Results Areas (RA): (i) RA1: Increasing the investment capacity of the Municipality of Casablanca; (ii) RA2: Improving the urban environment and access to basic services in the Program Area; and (iii) RA3: Improving the business environment in the Municipality of Casablanca.
